In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

In [2]:
def power_law(x, a, b):
    return a * np.power(x, b)

def exponential(x, a, b):
    return a * np.exp(b * x)

def stretched_exponential(x, a, b, c):
    return a * np.exp(-np.power(b * x, c))

In [3]:
basedir = Path('../analysis/')
dst_dirs = [str(p) for p in basedir.glob('distance*.csv') if p.is_file()]
bac_length = 4
dst_dirs

['../analysis/distance.csv']

In [4]:
def calc_params(df, shape, f):
    trv = df[df['shape']==shape]['trv']
    ct, bins = np.histogram(trv, bins=np.arange(0, np.ceil(df['trv'].max())), density=False)
    cum = np.cumsum(ct[::-1])
    rev_bin = bins[::-1]
    x = rev_bin[:-1]
    y = cum
    params_power, _ = curve_fit(power_law, x, y, p0=[1, -1])
    params_exp, _ = curve_fit(exponential, x, y, p0=[1, 0.01])
    params_stretch, _ = curve_fit(stretched_exponential, x, y, p0=[1, 0.01, 1], 
                                bounds=([0, 0, 0], [np.inf, np.inf, 10]), maxfev=5000)
    x_fit = np.linspace(min(x), max(x), 500)
    y_power_fit = power_law(x_fit, *params_power)
    y_exp_fit = exponential(x_fit, *params_exp)
    y_stretch_fit = stretched_exponential(x_fit, *params_stretch)
    y_power_pred = power_law(x, *params_power)
    y_exp_pred = exponential(x, *params_exp)
    y_stretch_pred = stretched_exponential(x, *params_stretch)
    r2_power = r2_score(y, y_power_pred)
    r2_exp = r2_score(y, y_exp_pred)
    r2_stretch = r2_score(y, y_stretch_pred)
    plt.scatter(x, y, label="Data", color="black", s=10)
    plt.plot(x_fit, y_power_fit, label=f"Power Law: $R^2$ = {r2_power:.3f}", linewidth=2)
    plt.plot(x_fit, y_exp_fit, label=f"Exponential: $R^2$ = {r2_exp:.3f}", linewidth=2, linestyle="--")
    plt.plot(x_fit, y_stretch_fit, label=f"Stretched Exp.: $R^2$ = {r2_stretch:.3f}", linewidth=2, linestyle=":")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"{f}: {len(trv)} trials")
    plt.legend()
    plt.savefig(f"../fit/{f}.png")
    plt.close()
    with open(f"../fit/{f}.params", 'w') as r:
        r.write(f"Power Law: {params_power}\n")
        r.write(f"Exponential: {params_exp}\n")
        r.write(f"Stretched Exponential: {params_stretch}\n")



In [6]:
if not os.path.exists('../fit'):
    os.mkdir('../fit')

In [7]:
for f in dst_dirs:
    df = pd.read_csv(f)
    df['diam'] = [float(i.split('_')[2]) for i in df['name']]
    df['bondcoeff'] = [float(i.split('_')[1]) for i in df['name']]
    df['ete'] = df['ete']/df['diam']/bac_length
    df['trv'] = df['trv']/df['diam']/bac_length
    calc_params(df, 'cur', 'curved_modelfit')
    calc_params(df, 'str', 'straight_modelfit')

grab all params

In [7]:
basedir = Path('../fit/')
dst_dirs = [str(p) for p in basedir.glob('*.params') if p.is_file()]
dst_dirs

['../fit/straight_modelfit.params', '../fit/curved_modelfit.params']

In [8]:
read_params = {}
for p in dst_dirs:
    with open(p, 'r') as f:
        read_params[p] = f.readlines()

In [9]:
# p.split('_')[1].split('.csv')[0]

'modelfit.params'

In [10]:
read_params

{'../fit/straight_modelfit.params': ['Power Law: [ 1.53051709e+03 -3.75822716e-01]\n',
  'Exponential: [ 1.00121993e+03 -1.94905355e-02]\n',
  'Stretched Exponential: [9.78605491e+02 1.89314691e-02 1.05262921e+00]\n'],
 '../fit/curved_modelfit.params': ['Power Law: [ 1.53591860e+03 -3.04141484e-01]\n',
  'Exponential: [ 1.05358380e+03 -1.42978761e-02]\n',
  'Stretched Exponential: [9.62506492e+02 1.31584783e-02 1.30047547e+00]\n']}

end
----